In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import string
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [29]:
df = pd.read_csv("D:/Projects/MLTask/test_data.csv")


In [30]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
pd.options.display.max_rows = 1000
df = df.assign(is_greet=0, is_bye=0, name=0, company=0)


Тут мы размечаем необходимые для выполнения условия реплики

In [32]:
greets = [1,2, 109,110,165, 166,249,250,251, 358, 304] 
byes = [108,163, 300, 301, 335, 336, 478, 479]
names = [3, 111,167, 251, 338]
companies = [3, 111, 251]
for i in greets:
    df.at[i, 'is_greet'] = 1
for i in byes:
    df.at[i, 'is_bye'] = 1
for i in names:
    df.at[i, 'name'] = 1
for i in companies:
    df.at[i, 'company'] = 1
df

,dlg_id,line_n,role,text,is_greet,is_bye,name,company
0,0,0,client,Алло,0,0,0,0
1,0,1,manager,Алло здравствуйте,1,0,0,0
2,0,2,client,Добрый день,1,0,0,0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,0,0,1,1
4,0,4,client,Ага,0,0,0,0
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,0,0,0,0
6,0,6,client,Да мы работаем с компанией которая нам подлива...,0,0,0,0
7,0,7,client,Как как бы уже до этого момента работаем все у...,0,0,0,0
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,0,0,0,0
9,0,9,client,Как бы нет,0,0,0,0


In [33]:
r_stopwords = stopwords.words("russian")
r_stopwords.remove('до') # чтобы не ломало прощание "до свидания"

In [34]:
def preproc(replica):
    nopunc = [char for char in replica if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    no_bad_words = [word for word in nopunc.split() if word.lower() not in r_stopwords]
    lemm_words = [lemmatizer.lemmatize(word) for word in no_bad_words]
    lemm_words = ' '.join(lemm_words)
    return lemm_words


In [35]:
df["text"] = df["text"].apply(lambda replica: preproc(replica))

In [36]:

data = df.copy()
y = data['is_greet'].values
data.drop(['is_greet'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, stratify=y)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (336, 7) (336,)
Test data: (144, 7) (144,)


In [37]:

vectorizer = TfidfVectorizer(min_df=2)

X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])

In [38]:

clf = GradientBoostingClassifier()
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf) #prediction from model
print('f1-score: ', f1_score(y_test, y_pred)) 
data = X_test.assign(is_greet=y_pred)
data[data["is_greet"] == 1]

f1-score:  1.0


,dlg_id,line_n,role,text,is_bye,name,company,is_greet
166,2,2,manager,Алло здравствуйте,0,0,0,1
2,0,2,client,Добрый день,0,0,0,1
304,4,2,client,Добрый,0,0,0,1


Видно, что наша модель нашла все приветствия на тестовой выборке

Теперь для is_bye

In [39]:
data = df.copy()
y = data['is_bye'].values
data.drop(['is_bye'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, stratify=y)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (360, 7) (360,)
Test data: (120, 7) (120,)


In [40]:
vectorizer = TfidfVectorizer(min_df=2)

X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])


In [41]:
from sklearn.ensemble import BaggingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf) 
print('f1-score: ', f1_score(y_test, y_pred)) 
data = X_test.assign(is_bye=y_pred)
data[data["is_bye"] == 1]

f1-score:  1.0


,dlg_id,line_n,role,text,is_greet,name,company,is_bye
301,3,52,client,до свидания,0,0,0,1
479,5,142,manager,до свидания хорошего вечера,0,0,0,1


Для имени менеджера

In [42]:
data = df.copy()
y = data['name'].values
data.drop(['name'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.20, stratify=y)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (384, 7) (384,)
Test data: (96, 7) (96,)


In [43]:
vectorizer = TfidfVectorizer(min_df=2)

X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])


In [44]:

clf = GradientBoostingClassifier()
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf) 
print('f1-score: ', f1_score(y_test, y_pred)) 
data = X_test.assign(name=y_pred)
data[data["name"] == 1]

f1-score:  1.0


,dlg_id,line_n,role,text,is_greet,is_bye,company,name
111,1,2,manager,зовут ангелина компания диджитал бизнес звоню ...,0,0,1,1


Для имени компании

In [45]:
data = df.copy()
y = data['name'].values
data.drop(['name'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, stratify=y)

print("Train data:",  X_train.shape, y_train.shape)
print("Test data:",  X_test.shape, y_test.shape)

Train data: (336, 7) (336,)
Test data: (144, 7) (144,)


In [46]:
vectorizer = TfidfVectorizer(min_df=2)

X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])

In [47]:
from sklearn.ensemble import BaggingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf) #prediction from model
print('f1-score: ', f1_score(y_test, y_pred)) 
data = X_test.assign(name=y_pred)
data[data["name"] == 1]

f1-score:  1.0


,dlg_id,line_n,role,text,is_greet,is_bye,company,name
167,2,3,manager,зовут ангелина компания диджитал бизнес звоню ...,0,0,0,1


Теперь проверим, что в каждом диалоге есть приветствие и прощание

In [48]:
# df["dlg_id"].unique()
hi = df[df["role"] == "manager"].groupby(by='dlg_id')['is_greet'].sum()
bye = df[df["role"] == "manager"].groupby(by='dlg_id')['is_bye'].sum()
dials = np.logical_and(hi, bye)
print("В диалгогах с этими номерами, менеджер поздоровался и попрощался:", *dials[dials].index)

В диалгогах с этими номерами, менеджер поздоровался и попрощался: 0 1 3


Под конец хотелось бы заметить, что данные модели всегда хорошо работают, так как данных по приветствию\прощанию не очень много, и они не успевают нормально научиться(